In [3]:
import math
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.nn as nn


# Creating the Neural network

In [4]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__() # Way to define a neural network in PyTorch
        self.net=torch.nn.Sequential(
            nn.Linear(2,20), # first layer, takes in 2 inputs and outputs 20
            nn.Tanh(), # activation function (tanh) 
            nn.Linear(20,30),  # second layer, takes in 20 inputs and outputs 30
            nn.Tanh(),
            nn.Linear(30,30),
            nn.Tanh(),
            nn.Linear(30,20),
            nn.Tanh(),
            nn.Linear(20,20),
            nn.Tanh(),
            nn.Linear(20,1) # output layer, takes in 20 inputs and outputs 1
            # total 7 layers (5 hidden layers, 1 input layer, 1 output layer)

        )
    def forward(self, x):
        return self.net(x)



# Initial and boundary conditions

In [9]:
class Net:
    def __init__(self):
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model=NN().to(device)  # neural network model  
        self.h=0.1 # step size for defiing the grid, step size for x
        self.k=0.1 # step size for defining the grid, step size for t
        x=torch.arange(-1,1+self.h,self.h)
        t=torch.arange(0,1+self.k,self.k)
        self.X=torch.stack(torch.meshgrid(x,t),dim=2).reshape(2,-1).T # defining the grid
        bc1=torch.stack(torch.meshgrid(x[0],t),dim=2).reshape(2,-1).T # boundary condition 1 (x=-1) 
        bc2=torch.stack(torch.meshgrid(x[-1],t),dim=2).reshape(2,-1).T # boundary condition 2 (x=1)
        ic=torch.stack(torch.meshgrid(x,t[0]),dim=2).reshape(2,-1).T # initial condition

        self.X_train=torch.cat([bc1,bc2,ic]) # training data
        y_bc1=torch.zeros(len(bc1)) # boundary condition 1 (x=-1)
        y_bc2=torch.zeros(len(bc2))
        y_ic=-torch.sin(math.pi*ic[:,0])
        self.y_train=torch.cat([y_bc1,y_bc2,y_ic])
        self.y_train=self.y_train.unsqueeze(1) # adding an extra dimension to y_train to make it a 2D tensor (required by PyTorch) 
        self.X=self.X.to(device) # moving the grid to the device (GPU)
        self.y_train=self.y_train.to(device)
        self.X_train=self.X_train.to(device)
        self.X.requires_grad=True # required by PyTorch to compute the gradient of X with respect to y (required by the loss function) 

        # Optimizer setting
        self.adam=torch.optim.Adam(self.model.parameters()) # Adam optimizer
        self.optimizer=torch.optim.LBFGS(
            self.model.parameters(),
            lr=1.0,
            max_iter=50000,
            max_eval=50000,
            history_size=50, # number of updates stored in the memory
            tolerance_grad=1.0*np.finfo(float).eps, # tolerance for the gradient
            line_search_fn="strong_wolfe" # strong Wolfe condition for line search 
        ) # L-BFGS optimizer 

        self.crierion=torch.nn.MSELoss() # Mean Squared Error loss function
        self.iter=1 # iteration counter 

    def loss_func(self):
        self.adam.zero_grad() # zero the gradients 
        self.optimizer.zero_grad() 

        y_pred=self.model(self.X_train) # prediction
        loss_data=self.crierion(y_pred,self.y_train) # loss function 
        u=self.model(self.X) # prediction 
        du_dX=torch.autograd.grad(u,self.X,grad_outputs=torch.ones_like(u),create_graph=True,retain_graph=True)[0] # gradient of u with respect to X 
        du_dx=du_dX[:,0]
        du_dt=du_dX[:,1]
        du_dXX=torch.autograd.grad(du_dX,self.X,grad_outputs=torch.ones_like(du_dX),create_graph=True,retain_graph=True)[0]
        du_dxx=du_dXX[:,0]
        loss_pde=self.criterion(du_dt+u.squeeze()*du_dx-(0.01/math.pi)*du_dxx,torch.zeros_like(du_dxx)) # loss function for the PDE
        loss=loss_pde+loss_data
        loss.backward() # backpropagation
        if self.iter%100==0:
            print('Iter:',self.iter,'Loss:',loss.item())
        self.iter+=1 # iteration counter

        return loss
    
    def train(self):
        self.model.train() # training mode
        for i in range(1000):
            self.adam.step(self.loss_func) # Adam optimizer
        self.optimizer.step(self.loss_func) # L-BFGS optimizer

    def eval_(self):
        self.model.eval()

 


# Training

In [10]:
net=Net()
net.train()

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.